In [2]:
import pandas as pd
import numpy as np

### Hyperpartisan

In [3]:
# res_df[(res_df.from_pretrained == 'bert-base-cased') & (res_df.input_seq_len == 818)]

In [6]:
df = pd.read_csv('results/hyperpartisan_baselines.csv')
df['input_seg_size'] = df.input_seq_len

In [9]:
pd.set_option('display.max_rows', 250)

In [11]:
res_df = df
gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'lr', 'lr_scheduler', 'iters']
metric_cols = ['f1', 'best_valid_f1']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['f1_std'] = res_df.groupby(gb_cols).std()[['f1']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['f1']
group_df['runs'] = res_df.groupby(gb_cols).agg({'f1':tuple}).f1.apply(lambda x: [round(i,2) for i in x])
group_df

f1  \
from_pretrained                    input_seg_size input_seq_len lr      lr_scheduler         iters             
bert-base-cased                    256            256           0.00001 constant_with_warmup 1000   0.897603   
                                                                        linear               1000   0.887320   
                                                                0.00005 constant_with_warmup 1000   0.895928   
                                                                        linear               1000   0.860399   
                                                                0.00010 constant_with_warmup 1000   0.893055   
                                                                        linear               1000   0.894268   
                                   512            512           0.00001 constant_with_warmup 1000   0.912609   
                                                                        linear               1000   0.910256   
                                                                0.00005 constant_with_warmup 1000   0.886067   
                                                                        linear               1000   0.916038   
                                                                0.00010 constant_with_warmup 1000   0.905875   
                                                                        linear               1000   0.910256   
bert-large-cased                   512            512           0.00001 constant_with_warmup 1000   0.885654   
                                                                        linear               1000   0.901679   
                                                                0.00005 constant_with_warmup 1000   0.895928   
                                                                        linear               1000   0.875002   
                                                                0.00010 constant_with_warmup 1000   0.801314   
                                                                        linear               1000   0.799157   
facebook/bart-base                 256            256           0.00001 constant_with_warmup 1000   0.941176   
                                                                        linear               1000   0.696296   
                                                                0.00005 constant_with_warmup 1000   0.922071   
                                                                        linear               1000   0.920738   
                                                                0.00010 constant_with_warmup 1000   0.911466   
                                                                        linear               1000   0.923077   
                                   512            512           0.00001 constant_with_warmup 1000   0.813853   
                                                                        linear               1000   0.846473   
                                                                0.00005 constant_with_warmup 1000   0.935897   
                                                                        linear               1000   0.907872   
                                                                0.00010 constant_with_warmup 1000   0.936623   
                                                                        linear               1000   0.949918   
                                   1024           1024          0.00001 constant_with_warmup 1000   0.823280   
                                                                        linear               1000   0.789418   
                                                                0.00005 constant_with_warmup 1000   0.901099   
                                                                        linear               1000   0.936623   
                                                                0.00010 co

In [5]:
res_df = pd.concat((pd.read_csv('results/hyp_grid.csv'), pd.read_csv('results/hyp.csv'), pd.read_csv('results/results_sbc/hyp_grid.csv')))#, df))
res_df.input_seg_size = res_df.input_seg_size.fillna(512)

In [125]:
models = ['bert-base-cased', 'roberta-base', 'microsoft/deberta-v3-base', 't5-base']
res_df = res_df[res_df.from_pretrained.isin(models)]
df = df[df.from_pretrained.isin(models)]

In [126]:
res_df.num_mem_tokens = res_df.num_mem_tokens.fillna(0)

In [127]:
pd.set_option('display.max_rows', 250)

In [128]:
res_df['n_segments'] = (res_df.input_seq_len / (res_df.input_seg_size - 3)).astype(int)

In [129]:
gb_cols = ['from_pretrained', 'input_seg_size', 'n_segments', 'num_mem_tokens', 'lr', 'lr_scheduler']
metric_cols = ['f1']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['f1_std'] = res_df.groupby(gb_cols).std()[['f1']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['f1']
group_df['runs'] = res_df.groupby(gb_cols).agg({'f1':tuple}).f1.apply(lambda x: [round(i,2) for i in x])
# group_df

In [130]:
gb_res_base = df.groupby(['from_pretrained', 'input_seg_size', 'lr', 'lr_scheduler']).mean()[['f1']].reset_index()
best_f1_idx_base = gb_res_base.groupby(['from_pretrained', 'input_seg_size']).transform(max)['f1'] == gb_res_base['f1']
# gb_res_base[best_f1_idx_base]

In [131]:
gb_res = res_df.groupby(['from_pretrained', 'input_seg_size', 'lr', 'lr_scheduler', 'num_mem_tokens', 'input_seq_len']).mean()[['f1']].reset_index()
best_f1_idx = gb_res.groupby(['from_pretrained', 'input_seg_size']).transform(max)['f1'] == gb_res['f1']
# gb_res[best_f1_idx]

In [132]:
df1 = gb_res_base[best_f1_idx_base]
df2 = gb_res[best_f1_idx]
# df2[join_cols]

In [133]:
join_cols = ['from_pretrained', 'input_seg_size']
pd.merge(df1, df2, how='left', left_on=join_cols, right_on=join_cols, suffixes=['_base', '_mem'])

,from_pretrained,input_seg_size,lr_base,lr_scheduler_base,f1_base,lr_mem,lr_scheduler_mem,num_mem_tokens,input_seq_len,f1_mem
0,bert-base-cased,256,0.00001,constant_with_warmup,0.897603,0.00001,linear,10.0,486.0,0.923077
1,bert-base-cased,256,0.00001,constant_with_warmup,0.897603,0.00001,linear,10.0,998.0,0.923077
2,bert-base-cased,512,0.00005,linear,0.916038,0.00005,constant_with_warmup,10.0,1996.0,0.943396
3,microsoft/deberta-v3-base,256,0.00001,constant_with_warmup,0.921046,0.00005,linear,10.0,486.0,0.981818
4,microsoft/deberta-v3-base,512,0.00005,linear,0.941679,0.00001,linear,10.0,998.0,0.967814
5,roberta-base,256,0.00005,linear,0.920738,0.00001,constant_with_warmup,10.0,486.0,0.933237
6,roberta-base,512,0.00001,constant_with_warmup,0.948685,0.00001,constant_with_warmup,10.0,998.0,0.981132
7,roberta-base,512,0.00001,constant_with_warmup,0.948685,0.00001,linear,10.0,1996.0,0.981132
8,t5-base,256,0.00010,constant_with_warmup,0.911224,NaN,NaN,NaN,NaN,NaN
9,t5-base,512,0.00005,constant_with_warmup,0.949918,0.00005,constant_with_warmup,10.0,2004.0,0.972048


In [139]:
# gb_res_by_train

In [140]:
gb_res_by_train = res_df.groupby(['from_pretrained', 'input_seg_size', 'lr', 'lr_scheduler', 'num_mem_tokens', 'input_seq_len']).mean()[['f1']].reset_index()
best_f1_idx_by_train = gb_res_by_train.groupby(['from_pretrained', 'input_seg_size', 'lr', 'lr_scheduler', 'input_seq_len']).transform(max)['f1'] == gb_res_by_train['f1']
df3 = gb_res_by_train[best_f1_idx_by_train]

join_cols = ['from_pretrained', 'input_seg_size', 'lr', 'lr_scheduler']
pd.merge(df1, df3, how='left', left_on=join_cols, right_on=join_cols, suffixes=['_base', '_mem'])

,from_pretrained,input_seg_size,lr,lr_scheduler,f1_base,num_mem_tokens,input_seq_len,f1_mem
0,bert-base-cased,256,0.00001,constant_with_warmup,0.897603,10.0,486.0,0.884615
1,bert-base-cased,256,0.00001,constant_with_warmup,0.897603,10.0,998.0,0.901961
2,bert-base-cased,512,0.00005,linear,0.916038,10.0,998.0,0.901961
3,bert-base-cased,512,0.00005,linear,0.916038,10.0,1497.0,0.923077
4,bert-base-cased,512,0.00005,linear,0.916038,10.0,1996.0,0.941176
5,microsoft/deberta-v3-base,256,0.00001,constant_with_warmup,0.921046,10.0,486.0,0.981132
6,microsoft/deberta-v3-base,512,0.00005,linear,0.941679,10.0,998.0,0.962963
7,microsoft/deberta-v3-base,512,0.00005,linear,0.941679,10.0,1497.0,0.961538
8,roberta-base,256,0.00005,linear,0.920738,10.0,486.0,0.912519
9,roberta-base,512,0.00001,constant_with_warmup,0.948685,10.0,998.0,0.981132


In [40]:
group_df.reset_index().sort_values(['from_pretrained', 'input_seg_size', 'n_segments', 'f1'], ascending=False)

,from_pretrained,input_seg_size,n_segments,num_mem_tokens,lr,lr_scheduler,f1,f1_std,n_runs,runs
235,t5-base,1024.0,1,0.0,0.00010,constant_with_warmup,0.963183,0.017840,3,"[0.95, 0.98, 0.96]"
233,t5-base,1024.0,1,0.0,0.00005,constant_with_warmup,0.962963,0.000000,3,"[0.96, 0.96, 0.96]"
236,t5-base,1024.0,1,0.0,0.00010,linear,0.962497,0.018872,3,"[0.96, 0.98, 0.94]"
234,t5-base,1024.0,1,0.0,0.00005,linear,0.943862,0.033084,3,"[0.91, 0.96, 0.96]"
231,t5-base,1024.0,1,0.0,0.00001,constant_with_warmup,0.887464,0.002467,3,"[0.89, 0.89, 0.88]"
232,t5-base,1024.0,1,0.0,0.00001,linear,0.807892,0.015539,3,"[0.79, 0.82, 0.81]"
226,t5-base,512.0,3,10.0,0.00005,constant_with_warmup,0.972048,0.012847,2,"[0.98, 0.96]"
229,t5-base,512.0,3,10.0,0.00010,linear,0.962264,0.026683,2,"[0.94, 0.98]"
228,t5-base,512.0,3,10.0,0.00010,constant_with_warmup,0.953180,0.013836,2,"[0.94, 0.96]"
227,t5-base,512.0,3,10.0,0.00005,linear,0.877551,0.028861,2,"[0.86, 0.9]"


In [86]:
# model = 'bert-base-cased'
# # model = 'roberta-base'
# # model = 'google/electra-base-discriminator'
# # model = 'microsoft/deberta-v3-base'

# model = 't5-base'
# # model = 'facebook/bart-base'
# group_df.iloc[np.where(group_df.reset_index().from_pretrained == model)[0]]#.sort_values('f1', ascending=False)
# # group_df.iloc[np.where((group_df.reset_index().from_pretrained == model) & (group_df.reset_index().input_seg_size == 512))[0]]#.sort_values('f1', ascending=False)

In [125]:
res_df = pd.read_csv('results/hyp.csv')
res_df.input_seg_size = res_df.input_seg_size.fillna(512)
# gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'num_mem_tokens']
# metric_cols = ['f1', 'best_valid_f1', 'accuracy', 'best_valid_accuracy']
# group_df = res_df.groupby(gb_cols).mean()[metric_cols]
# group_df['f1_std'] = res_df.groupby(gb_cols).std()[['f1']]
# group_df['n_runs'] = res_df.groupby(gb_cols).count()['f1']
# group_df

In [121]:
# slice = res_df[(res_df.num_mem_tokens.isin({0, 10, 100})) & (res_df.input_seq_len > 256)]
# gb_cols = ['from_pretrained', 'input_seq_len', 'num_mem_tokens']
# group_df = slice.groupby(gb_cols).mean()[['f1', 'best_valid_f1', 'accuracy', 'best_valid_accuracy']].reset_index()
# group_df['n_segments'] = (group_df.reset_index().input_seq_len / 512).apply(lambda x: np.ceil(x)).astype(int).values
# group_df[['from_pretrained', 'n_segments', 'num_mem_tokens', 'f1', 'accuracy']]

### contract-nli

In [87]:
# res_df = pd.read_csv('results/cnli.csv')
res_df = pd.concat((pd.read_csv('results/cnli.csv'), pd.read_csv('results/results_sbc/cnli.csv')))
res_df.input_seg_size = res_df.input_seg_size.fillna(512)

In [88]:
res_df = res_df[res_df.best_valid_exact_match > 45]

In [90]:
gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'num_mem_tokens', 'lr', 'lr_scheduler']
metric_cols = ['best_valid_exact_match']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['std'] = res_df.groupby(gb_cols).std()[['best_valid_exact_match']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['best_valid_exact_match']
group_df['runs'] = res_df.groupby(gb_cols).agg({'best_valid_exact_match':tuple}).best_valid_exact_match.apply(lambda x: [round(i,2) for i in x])
group_df

best_valid_exact_match  \
from_pretrained           input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                                   
bert-base-cased           512.0          918           50             0.00001 constant_with_warmup               72.736031   
                                                                              linear                             71.387283   
                                                                      0.00005 constant_with_warmup               57.514450   
                                                                              linear                             68.400772   
                                         968           25             0.00001 constant_with_warmup               72.157997   
                                                                              linear                             74.084778   
                                                                      0.00005 constant_with_warmup               71.965317   
                                                                              linear                             71.676300   
                                         998           10             0.00001 constant_with_warmup               72.976879   
                                                                              linear                             71.387283   
                                                                      0.00005 constant_with_warmup               71.483624   
                                                                              linear                             71.724472   
                                                                      0.00010 constant_with_warmup               54.720615   
                                                                              linear                             59.730251   
                                         1377          50             0.00001 constant_with_warmup               71.387283   
                                                                              linear                             72.157997   
                                                                      0.00005 constant_with_warmup               57.032757   
                                                                              linear                             65.606934   
                                         1452          25             0.00001 constant_with_warmup               71.676300   
                                                                              linear                             72.350677   
                                                                      0.00005 constant_with_warmup               65.125244   
                                                                              linear                             57.803467   
                                         1497          10             0.00001 constant_with_warmup               68.063583   
                                                                              linear                             73.410408   
                                                                      0.00005 constant_with_warmup               57.321774   
                                                                              linear                             57.321774   
                                                                      0.00010 constant_with_warmup               54.816956   
                                                                              linear                             50.096340   
                                         1836          50             0.00001 constant_with_warmup               71.772636   
                                                                              linear                             71.001930   
                                                                    

In [58]:
# model = 'bert-base-cased'
# model = 'roberta-base'
model = 'microsoft/deberta-v3-base'

# model = 't5-base'
# model = 'facebook/bart-base'
group_df.iloc[np.where(group_df.reset_index().from_pretrained == model)[0]]

best_valid_exact_match  \
from_pretrained           input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                                   
microsoft/deberta-v3-base 512.0          998           10             0.00001 constant_with_warmup               73.365387   
                                                                              linear                             72.884613   
                                                                      0.00005 constant_with_warmup               59.903847   
                                                                              linear                             70.192307   
                                         1497          10             0.00005 constant_with_warmup               55.769230   
                                                                              linear                             56.730770   

                                                                                                    std  \
from_pretrained           input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                
microsoft/deberta-v3-base 512.0          998           10             0.00001 constant_with_warmup  NaN   
                                                                              linear                NaN   
                                                                      0.00005 constant_with_warmup  NaN   
                                                                              linear                NaN   
                                         1497          10             0.00005 constant_with_warmup  NaN   
                                                                              linear                NaN   

                                                                                                    n_runs  \
from_pretrained           input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                   
microsoft/deberta-v3-base 512.0          998           10             0.00001 constant_with_warmup       1   
                                                                              linear                     1   
                                                                      0.00005 constant_with_warmup       1   
                                                                              linear                     1   
                                         1497          10             0.00005 constant_with_warmup       1   
                                                                              linear                     1   

                                                                                                       runs  
from_pretrained           input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                   
microsoft/deberta-v3-base 512.0          998           10             0.00001 constant_with_warmup  [73.37]  
                                                                              linear                [72.88]  
                                                                      0.00005 constant_with_warmup   [59.9]  
                                                                              linear                [70.19]  
                                         1497          10             0.00005 constant_with_warmup  [55.77]  
                                                                              linear                [56.73]

In [175]:

# res_df.groupby(gb_cols).agg({'best_valid_exact_match':tuple})

In [84]:
# gb_cols = ['from_pretrained', 'input_seq_len', 'num_mem_tokens']
# group_df = res_df.groupby(gb_cols).mean()[['best_valid_exact_match']]

# group_df['n_segments'] = (group_df.reset_index().input_seq_len / 512).apply(lambda x: np.ceil(x)).astype(int).values
# # group_df['f1_std'] = res_df.groupby(gb_cols).std()[['f1']]
# # group_df['n_runs'] = res_df.groupby(gb_cols).count()['f1']
# group_df[['n_segments', 'best_valid_exact_match']]

### quality

In [94]:
# res_df = pd.read_csv('results/cnli.csv')
res_df = pd.concat((pd.read_csv('results/results_sbc/quality.csv'),))
res_df.input_seg_size = res_df.input_seg_size.fillna(512)

In [97]:
res_df = res_df[res_df.best_valid_exact_match > 10]

In [99]:
gb_cols = ['from_pretrained', 'input_seg_size', 'input_seq_len', 'num_mem_tokens', 'lr', 'lr_scheduler']
metric_cols = ['best_valid_exact_match']
group_df = res_df.groupby(gb_cols).mean()[metric_cols]
group_df['std'] = res_df.groupby(gb_cols).std()[['best_valid_exact_match']]
group_df['n_runs'] = res_df.groupby(gb_cols).count()['best_valid_exact_match']
group_df['runs'] = res_df.groupby(gb_cols).agg({'best_valid_exact_match':tuple}).best_valid_exact_match.apply(lambda x: [round(i,2) for i in x])
group_df

best_valid_exact_match  \
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                                   
t5-base         512.0          1001          10             0.00001 constant_with_warmup               21.206837   
                                                                    linear                             19.109525   
                                                            0.00005 constant_with_warmup               24.387057   
                                                                    linear                             23.561837   

                                                                                               std  \
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                     
t5-base         512.0          1001          10             0.00001 constant_with_warmup  1.605965   
                                                                    linear                1.593030   
                                                            0.00005 constant_with_warmup  0.614343   
                                                                    linear                0.507987   

                                                                                          n_runs  \
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                   
t5-base         512.0          1001          10             0.00001 constant_with_warmup       8   
                                                                    linear                     8   
                                                            0.00005 constant_with_warmup       7   
                                                                    linear                     8   

                                                                                                                                       runs  
from_pretrained input_seg_size input_seq_len num_mem_tokens lr      lr_scheduler                                                             
t5-base         512.0          1001          10             0.00001 constant_with_warmup  [21.14, 21.48, 22.67, 21.76, 21.81, 21.38, 22....  
                                                                    linear                [19.13, 17.31, 18.79, 19.51, 21.86, 20.04, 16....  
                                                            0.00005 constant_with_warmup  [25.12, 25.07, 23.44, 24.02, 24.74, 24.11, 24.21]  
                                                                    linear                [23.49, 22.58, 23.2, 23.44, 24.02, 24.16, 23.8...